

## Extension 01 // Neural Network CNN

### Machine Learning Algorithm: Neural Networks

The base strategy we are going to start with is Neural Networks. This topic was was covered in the week 8. In this first section I will include the code given in homework 8. The performance will give us a benchmark we can work from. 

Our extension to the code in homework 8 will be to add a convolutional layers to the neural network. First, we will use a PyTorch implementation. Then, we will write our own implementation using Numpy. 

Here is the Neural Network implemented in homework 8. 


In [0]:
# set Global Hyperparameters 
epochs = 10

## Load Both MNIST and CIFAR Dataset to temp-data in Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from pathlib import Path
import os
import requests
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

os.chdir('/content/drive/My Drive')
print(os.getcwd())

# Move into primary Drive Folder
os.chdir('/content/drive/My Drive')

# Make temp-data directory and move inside
path_name = 'temp-data/'
Path("temp-data/").mkdir(parents=True, exist_ok=True)
os.chdir(path_name)
print('\nData directory created...')

print('Download starting...')
url = 'https://github.com/aobject/public-nyu-ml/blob/master/ML-Project/Data/cifar-10.p?raw=true'
target_path = 'cifar-10.p'
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

url = 'https://github.com/aobject/public-nyu-ml/blob/master/ML-Project/Data/mnist.p?raw=true'
target_path = 'mnist.p'
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())
print('\nThe files in the data folder include:')
path = '.'
files = os.listdir(path)
for name in files:
    print(name)

digits = unpickle('mnist.p')
print('\nDigits data shape is {}'.format(digits['data'].shape))
cifar = unpickle('cifar-10.p')
cifar[b'data'] = cifar[b'data'][:1000]  # mabye increase to 1800 later
cifar[b'labels'] = cifar[b'labels'][:1000]
print('cifar data shape is {}'.format(cifar[b'data'].shape))

print('\nAll done, your data is ready to go!!!\n')

/content/drive/My Drive

Data directory created...
Download starting...

The files in the data folder include:
mnist.p
cifar-10.p

Digits data shape is (1797, 64)
cifar data shape is (1000, 3072)

All done, your data is ready to go!!!



## Part A ( i ) // Neural Network Without CNN from Homework 8 Notebook on MNIST Dataset I: 
## Neural Network From the Homework Notebook

The base strategy we are going to start with is Neural Networks. This topic was was covered in the week 8. In this first section I will include the code given in homework 8. The performance will give us a benchmark we can work from. 

Our extension to the code in homework 8 will be to add a convolutional layers to the neural network. First, we will use a PyTorch implementation. Then, we will write our own implementation. 

Here is the Neural Network implemented in homework 8. 

In [4]:
# Import Libraries
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import accuracy_score # 
import numpy as np
import numpy.random as r # We will randomly initialize our weights
import matplotlib.pyplot as plt 
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Legend
from bokeh.io import output_notebook
output_notebook()

X = digits['data']
y = digits['target']
print("The shape of the digits dataset:") 
print(X.shape)

X_scale = StandardScaler()
X = X_scale.fit_transform(digits.data)

X[0,:] # Looking the new features after scaling

#Split the data into training and test set.  60% training and %40 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

def convert_y_to_vect(y):
    y_vect = np.zeros((len(y), 10))
    for i in range(len(y)):
        y_vect[i, y[i]] = 1
    return y_vect


# convert digits to vectors
y_v_train = convert_y_to_vect(y_train)
y_v_test = convert_y_to_vect(y_test)

def f(z):
    return 1 / (1 + np.exp(-z))


def f_deriv(z):
    return f(z) * (1 - f(z))

def setup_and_init_weights(nn_structure):
    W = {} #creating a dictionary i.e. a set of key: value pairs
    b = {}
    for l in range(1, len(nn_structure)):
        W[l] = r.random_sample((nn_structure[l], nn_structure[l-1])) #Return “continuous uniform” random floats in the half-open interval [0.0, 1.0). 
        b[l] = r.random_sample((nn_structure[l],))
    return W, b

def init_tri_values(nn_structure):
    tri_W = {}
    tri_b = {}
    for l in range(1, len(nn_structure)):
        tri_W[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
        tri_b[l] = np.zeros((nn_structure[l],))
    return tri_W, tri_b

def feed_forward(x, W, b):
    a = {1: x} # create a dictionary for holding the a values for all levels
    z = { } # create a dictionary for holding the z values for all the layers
    for l in range(1, len(W) + 1): # for each layer
        node_in = a[l]
        z[l+1] = W[l].dot(node_in) + b[l]  # z^(l+1) = W^(l)*a^(l) + b^(l)
        a[l+1] = f(z[l+1]) # a^(l+1) = f(z^(l+1))
    return a, z

def calculate_out_layer_delta(y, a_out, z_out):
    # delta^(nl) = -(y_i - a_i^(nl)) * f'(z_i^(nl))
    return -(y-a_out) * f_deriv(z_out) 


def calculate_hidden_delta(delta_plus_1, w_l, z_l):
    # delta^(l) = (transpose(W^(l)) * delta^(l+1)) * f'(z^(l))
    return np.dot(np.transpose(w_l), delta_plus_1) * f_deriv(z_l)

def predict_y(W, b, X, n_layers):
    N = X.shape[0]
    y = np.zeros((N,))
    for i in range(N):
        a, z = feed_forward(X[i, :], W, b)
        y[i] = np.argmax(a[n_layers])
    return y

def train_nn(nn_structure, X_train, y_train, X_test, y_test, iter_num=100, alpha=0.25):
    W, b = setup_and_init_weights(nn_structure)
    cnt = 0
    N = len(y_train)
    avg_cost_func = []

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []
    train_examples = len(y_train)
    test_examples = len(y_test)

    print('Starting gradient descent for {} epochs'.format(iter_num))
    while cnt < iter_num:
        print('Epoch {} of {}'.format(cnt, iter_num))
        tri_W, tri_b = init_tri_values(nn_structure)
        avg_cost = 0

        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0


        for i in range(N):
            delta = {}
            # perform the feed forward pass and return the stored a and z values, to be used in the
            # gradient descent step
            a, z = feed_forward(X_train[i, :], W, b)
            # loop from nl-1 to 1 backpropagating the errors
            for l in range(len(nn_structure), 0, -1):
                if l == len(nn_structure):
                    delta[l] = calculate_out_layer_delta(y_train[i,:], a[l], z[l])
                    loss_train += np.linalg.norm((y_train[i,:]-a[l]))
                    if np.argmax(a[len(nn_structure)]) == np.argmax(y_v_train[i]):
                      acc_train += 1.0
                else:
                    if l > 1:
                        delta[l] = calculate_hidden_delta(delta[l+1], W[l], z[l])
                    # triW^(l) = triW^(l) + delta^(l+1) * transpose(a^(l))
                    tri_W[l] += np.dot(delta[l+1][:,np.newaxis], np.transpose(a[l][:,np.newaxis]))# np.newaxis increase the number of dimensions
                    # trib^(l) = trib^(l) + delta^(l+1)
                    tri_b[l] += delta[l+1]
        # perform the gradient descent step for the weights in each layer
        for l in range(len(nn_structure) - 1, 0, -1):
            W[l] += -alpha * (1.0/N * tri_W[l])
            b[l] += -alpha * (1.0/N * tri_b[l])
        
        # Test model

        # get y prediction
        # y_pred = predict_y(W, b, X_test, 3)
        
        for i in range(test_examples):
         
            a, z = feed_forward(X_test[i, :], W, b)
            # calculate loss
            loss_test += np.linalg.norm((y_test[i,:]-a[len(nn_structure)]))
            # calculate accuracy
            if np.argmax(a[len(nn_structure)]) == np.argmax(y_v_test[i]):
                acc_test += 1
            
        # complete the average cost calculation
        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)
        cnt += 1
    return W, b, (loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)


nn_structure = [64, 30, 10]
    
# train the NN
%time W, b, result_baseline_hwcode_mnist = train_nn(nn_structure, X_train, y_v_train, X_test, y_v_test, epochs, 0.25)

def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch'            : range(1, len(loss_test_seq) + 1),
        'train_loss'    : loss_train_seq,
        'test_loss'        :  loss_test_seq,
    })

    p = figure(title='MNIST Loss: NN without CNN provide in HW', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss' 

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Loss', '@train_loss{0.000 a}'),
                  ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_train_seq) + 1),
        'train_acc'    : np.array(acc_train_seq),
        'test_acc'        :  np.array(acc_test_seq),
    })

    p = figure(title='MNIST Accuracy: NN without CNN provided in HW', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy' 
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Accuracy', '@{train_acc}{%0.2f}'),
                  ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)


loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_baseline_hwcode_mnist






The shape of the digits dataset:
(1797, 64)
Starting gradient descent for 10 epochs
Epoch 0 of 10
Epoch 1 of 10
Epoch 2 of 10
Epoch 3 of 10
Epoch 4 of 10
Epoch 5 of 10
Epoch 6 of 10
Epoch 7 of 10
Epoch 8 of 10
Epoch 9 of 10
CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.16 s


In [5]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 8.205841446453407%


## Part A ( i ) Continued: A Better Baseline
## Baseline PyTorch Neural Network Without CNN on MNIST Dataset I: 

In [6]:
# Import modules
import torch
import numpy as np
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()

class MNISTDataset(Dataset):
    def __init__(self, data, label):
        self.data = data.reshape((-1,8,8,1))
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, item):

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = self.data[item].transpose((2, 0, 1))
        image = torch.from_numpy(image)
        target = self.label[item]
        target = torch.from_numpy(target)
        return (image, target)


class MNIST(nn.Module):

    # Our batch shape for input x is (1, 8, 8)

    def __init__(self):
        super(MNIST, self).__init__()

        # Input channels = 1, output channels = 18
        # self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
        # self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # self.fc1 = nn.Linear(18 * 8 * 8, 64)
        self.fc1 = nn.Linear(64, 30)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):

        # Activation of the first convolution
        # Size changes from (1, 8, 8) to (18, 8, 8)
        # x = F.relu(self.conv1(x))

        # x = self.pool(x)

        # Reshape data to input to the input layer of the neural net
        # Size changes from (18, 16, 16) to (1, 1152)
        x = x.view(-1, 8 * 8)

        # Computes the activation of the first fully connected layer
        # Size changes from (1, 1152) to (1, 64)
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))

        # x = self.fc2(x)
        # x = F.log_softmax(x, dim=1)

        return x


def scorecard(output, labels):
    score = 0
    score = np.where(np.argmax(output, axis=1) == np.argmax(labels, axis=1), 1, 0)
    # for i in range(len(output)):
    #     if np.argmax(output[i,:], axis=1) == np.argmax(labels[i,:], axis=1):
    #         score += 1
    score = np.sum(score)
    return score



def trainModel(digits, model, batch_size, num_epochs, learning_rate):
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", num_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)

    X = digits['data']
    y = digits['target']

    class Scaler():
      def __init__(self, training_data):
        self.u = np.mean(training_data)
        self.s = np.std(training_data)

      def scale_data(self, in_data):
        return (in_data - self.u) / self.s

    X_scale = Scaler(X)
    X = X_scale.scale_data(X)

    def split(in_X, in_y, percentage_test):
        data_size = len(in_y)
        order = np.arange(0, data_size, dtype=int)
        np.random.shuffle(order)
        out_X = in_X[order]
        out_y = np.array(in_y)[order].tolist()
        split = int(np.rint(data_size * percentage_test))
        return out_X[split:], out_X[0:split], out_y[split:], out_y[0:split]

    # Split the data into training and test set.  60% training and %40 test
    X_train, X_test, y_train, y_test = split(X, y, 0.4)

    def convert_y_to_vect(y):
        y_vect = np.zeros((len(y), 10))
        for i in range(len(y)):
            y_vect[i, y[i]] = 1
        return y_vect

    # convert digits to vectors
    y_v_train = convert_y_to_vect(y_train)
    y_v_test = convert_y_to_vect(y_test)

    train_dataset = MNISTDataset(X_train, y_v_train)
    test_dataset = MNISTDataset(X_test, y_v_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    torch.manual_seed(0)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(0)
    mean, std = (0.5,), (0.5,)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    model = model.double()

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []

    for i in range(num_epochs):
        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0
        train_examples = 0
        test_examples = 0

        for j, (images, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            acc_train += scorecard(output.detach().numpy(), labels.detach().numpy())
            train_examples += len(labels)
            loss_train += loss.detach().numpy()
            loss.backward()
            optimizer.step()


        for images, labels in test_loader:

            with torch.no_grad():
                output = model(images)

                loss = criterion(output, labels)
                loss_test += loss.detach().numpy()
                acc_test += scorecard(output.detach().numpy(), labels.detach().numpy())
                test_examples += len(labels)
        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)
        print('Epoch {} with test loss {} and test accuracy {}\n\n'.format(i, (loss_test / test_examples),(acc_test / test_examples) ))

    return (loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)


def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch'            : range(1, len(loss_test_seq) + 1),
        'train_loss'    : loss_train_seq,
        'test_loss'        :  loss_test_seq,
    })

    p = figure(title='MNIST Loss: Baseline without CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss' 

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Loss', '@train_loss{0.000 a}'),
                  ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_train_seq) + 1),
        'train_acc'    : np.array(acc_train_seq),
        'test_acc'        :  np.array(acc_test_seq),
    })

    p = figure(title='MNIST Accuracy: Baseline without CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy' 
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Accuracy', '@{train_acc}{%0.2f}'),
                  ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)

batch_size = 1
learning_rate = 0.01
num_epochs = epochs

model = MNIST()

%time result_baseline_pytorch_mnist = trainModel(digits, model, batch_size, num_epochs, learning_rate)
loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_baseline_pytorch_mnist





===== HYPERPARAMETERS =====
batch_size= 1
epochs= 10
learning_rate= 0.01
Epoch 0 with test loss 0.10901104598421028 and test accuracy 0.0458970792767733


Epoch 1 with test loss 0.0942172668452986 and test accuracy 0.08484005563282336


Epoch 2 with test loss 0.09113696263224312 and test accuracy 0.13073713490959665


Epoch 3 with test loss 0.09009772041456994 and test accuracy 0.18219749652294853


Epoch 4 with test loss 0.08962356710128933 and test accuracy 0.2364394993045897


Epoch 5 with test loss 0.08934737726536895 and test accuracy 0.26842837273991654


Epoch 6 with test loss 0.089149420876955 and test accuracy 0.3045897079276773


Epoch 7 with test loss 0.08898348343236896 and test accuracy 0.325452016689847


Epoch 8 with test loss 0.08882952992457946 and test accuracy 0.34492350486787204


Epoch 9 with test loss 0.08867805144216365 and test accuracy 0.3588317107093185


CPU times: user 8.92 s, sys: 527 ms, total: 9.45 s
Wall time: 9.61 s


In [7]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 35.88317107093185%


## Part A ( ii ) // Convolutional Neural Network Extension Using NumPy Only On MNIST Dataset I


In [8]:
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()


class CNN:

    def __init__(self, num_kernels, pool_size, out_nodes, seed):
        self.kernels = self.Kernels(num_kernels)  # 8x8x1  to 6x6x10
        # self.pool = self.Pool(pool_size)  # 6x6x10 to 3x3x10
        self.sigmoid = self.Sigmoid(6 * 6 * num_kernels, out_nodes)  # 6x6x10 to 10
        np.random.seed(seed)

    class Kernels:
        # The set of kernels and kernel functions

        def __init__(self, num_kernels):
            self.num_kernels = num_kernels
            self.size_kernel = 3
            self.kernels = np.random.randn(num_kernels, 3, 3) / (self.size_kernel ** 2)

        def forward(self, x):
            self.cur_x = x

            height, width = x.shape
            result = np.zeros((height - 2, width - 2, self.num_kernels))

            for i in range(height-2):
                for j in range(width-2):
                    block = x[i:(i+3), j:(j+3)]
                    result[i,j] = np.sum(block * self.kernels, axis=(1, 2))
            return result

        def back(self, delta_cnn, alpha):
            height, width = self.cur_x.shape
            dk = np.zeros(self.kernels.shape)
            delta_cnn = delta_cnn.reshape(height-2, width-2,self.num_kernels)
            for i in range(height - 2):
                for j in range(width - 2):
                    block_x = self.cur_x[i:(i+3), j:(j+3)]
                    # block_delta = delta_cnn[i:(i+3), j:(j+3)]
                    for k in range(self.num_kernels):
                        dk[k] += delta_cnn[i,j,k] * block_x

            self.kernels += -alpha * dk


    class Pool:
        # use max pooling with size 2x2
        def __init__(self, pool_size):
            self.dim = pool_size  # size of the pool

        def forward(self, x):
            self.cur_input = x
            height, width, num_k = x.shape
            result = np.zeros((height // 2, width // 2, num_k))
            for i in range(height // 2):
                for j in range(width // 2):
                    block = x[(i*2):(i*2+2), (j*2):(j*2+2)]
                    result[i,j] = np.amax(block, axis=(0,1))
            return result

        def back(self, delta_pool):

            height, width, num_k = self.cur_input.shape
            delta_pool = delta_pool.reshape((height//2, width//2, num_k))
            delta_cnn = np.zeros((height, width, num_k))

            for i in range(height // 2):
                for j in range(width // 2):
                    block = self.cur_input[(i*2):(i*2+2),(j*2):(j*2+2)]
                    b_h, b_w, b_k = block.shape
                    maximum = np.amax(block, axis=(0,1))
                    for s in range(b_h):
                        for t in range(b_w):
                            for u in range(b_k):
                                if block[s, t, u] == maximum[u]:
                                    delta_cnn[i*2+s, j*2+t, u] = delta_pool[i,j,u]
            return delta_cnn


    class Sigmoid:
        # Use a fully connected layer using sigmoid activation

        def __init__(self, in_nodes, out_nodes):
            self.w = np.random.randn(out_nodes, in_nodes) / in_nodes
            self.b = np.zeros((out_nodes, 1))

        def sig(self,z):
            return 1 / (1+np.exp(-z))

        def d_sig(self,z):
            return self.sig(z) * (1-self.sig(z))

        def loss(self, y, yhat):
            self.y_vector = np.zeros((len(yhat), 1))
            self.y_vector[y, 0] = 1.0
            loss = np.linalg.norm(self.y_vector - yhat)
            accuracy = 1 if np.argmax(yhat[:, 0]) == y else 0
            return loss, accuracy

        def forward(self, x):
            # TODO forward pass
            self.z_cnn = x.reshape(-1, 1)
            self.a_cnn = self.sig(self.z_cnn)

            self.z_out = np.matmul(self.w, self.a_cnn) + self.b
            self.a_out = self.sig(self.z_out)

            return self.a_out

        def back(self, alpha):

            w_grad = np.zeros(self.w.shape)
            b_grad = np.zeros(self.b.shape)

            # get delta for out layer
            delta_out = -(self.y_vector - self.a_out) * self.d_sig(self.z_out)

            # get delta for hidden layer
            delta_pool = np.dot(self.w.T, delta_out) * self.d_sig(self.z_cnn)

            w_grad += -alpha * np.matmul(delta_out, self.a_cnn.T)
            b_grad += -alpha * delta_out
            # TODO back pass through pool and cnn layers
            return delta_pool


def format_data(digits):
    X = digits['data']
    y = digits['target']

    class Scaler():
      def __init__(self, training_data):
        self.u = np.mean(training_data)
        self.s = np.std(training_data)

      def scale_data(self, in_data):
        return (in_data - self.u) / self.s

    X_scale = Scaler(X)
    X = X_scale.scale_data(X)

    def split(in_X, in_y, percentage_test):
        data_size = len(in_y)
        order = np.arange(0, data_size, dtype=int)
        np.random.shuffle(order)
        out_X = in_X[order]
        out_y = np.array(in_y)[order].tolist()
        split = int(np.rint(data_size * percentage_test))
        return out_X[split:], out_X[0:split], out_y[split:], out_y[0:split]

    # Split the data into training and test set.  60% training and %40 test
    X_train, X_test, y_train, y_test = split(X, y, 0.4)

    # Reshape X_train and X_test to be (-1, 8, 8)
    X_train = X_train.reshape((-1, 8, 8))
    X_test = X_test.reshape((-1, 8, 8))
    return (X_train, X_test, y_train, y_test)

def shuffle(X, y):
    data_size = len(y)
    order = np.arange(0, data_size, dtype=int)
    np.random.shuffle(order)
    new_order = np.random.permutation(len(y))
    X = X[new_order]
    y = np.array(y)[new_order].tolist()
    return X, y

def forward(x, y, model):
    result = model.kernels.forward(x)
    # result = model.pool.forward(result)
    yhat = model.sigmoid.forward(result)
    loss, accuracy = model.sigmoid.loss(y, yhat)
    return yhat, loss, accuracy


def train(x, y, model, alpha=0.1):
    # forward
    _, loss, accuracy = forward(x, y, model)

    # back
    delta = model.sigmoid.back(alpha)
    # delta = model.pool.back(delta)
    model.kernels.back(delta, alpha)

    return loss, accuracy


def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch'            : range(1, len(loss_test_seq) + 1),
        'train_loss'    : loss_train_seq,
        'test_loss'        :  loss_test_seq,
    })

    p = figure(title='MNIST Loss: With NumPy Sigmoid CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss'

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Loss', '@train_loss{0.000 a}'),
                  ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_train_seq) + 1),
        'train_acc'    : np.array(acc_train_seq),
        'test_acc'        :  np.array(acc_test_seq),
    })

    p = figure(title='MNIST Accuracy: With Numpy Sigmoid CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Accuracy', '@{train_acc}{%0.2f}'),
                  ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)

kernel_layers = 64
pool_size = 2
epochs = epochs
out_nodes = 10
alpha = 0.4
seed = 34

def train_cnn(digits, epochs, alpha, kernel_layers, pool_size, out_nodes, seed):

    X_train, X_test, y_train, y_test = format_data(digits)

    model = CNN(kernel_layers, pool_size, out_nodes, seed)

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []
    train_examples = len(y_train)
    test_examples = len(y_test)

    print('=' * 30)
    print('==== Hyperparameters ====')
    print('='*30)
    print('Learning Rate: {}'.format(alpha))
    print('Kernel Layers: {}'.format(kernel_layers))
    # print('Pool Size: {}'.format(pool_size))
    print('Epochs: {}'.format(epochs))
    print('Seed: {}'.format(seed))
    print('=' * 30 + '\n')
    for epoch in range(epochs):

        if epoch != 0:
            print('Test Loss: {}\nTest Accuracy: {} %'.format(loss_test_seq[-1], acc_test_seq[-1]*100))
            print('Train Loss: {}\nTrain Accuracy: {} %\n\n'.format(loss_train_seq[-1], acc_train_seq[-1]*100))
        print('=' * 30)
        print('Starting epoch {} of {}'.format(epoch, epochs))
        X_train, y_train = shuffle(X_train, y_train)

        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0

        for i, (image, label) in enumerate(zip(X_train, y_train)):
            loss, acc = train(image, label, model, alpha)
            loss_train += loss
            acc_train += acc


        for i, (image, label) in enumerate(zip(X_test, y_test)):
            _, loss, acc = forward(image, label, model)
            loss_test += loss
            acc_test += acc


        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)
    
    return loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq

% time result_mnist_cnn_numpy = train_cnn(digits, epochs, alpha, kernel_layers, pool_size, out_nodes, seed)
loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_mnist_cnn_numpy



==== Hyperparameters ====
Learning Rate: 0.4
Kernel Layers: 64
Epochs: 10
Seed: 34

Starting epoch 0 of 10
Test Loss: 1.5797426299470898
Test Accuracy: 15.02086230876217 %
Train Loss: 1.5812206142200183
Train Accuracy: 13.636363636363635 %


Starting epoch 1 of 10
Test Loss: 1.5774945432792333
Test Accuracy: 19.19332406119611 %
Train Loss: 1.5786983785873683
Train Accuracy: 15.955473098330241 %


Starting epoch 2 of 10
Test Loss: 1.5757488113566616
Test Accuracy: 18.915159944367176 %
Train Loss: 1.57669653634247
Train Accuracy: 18.460111317254174 %


Starting epoch 3 of 10
Test Loss: 1.5744001383372113
Test Accuracy: 18.636995827538247 %
Train Loss: 1.5751540317042245
Train Accuracy: 19.387755102040817 %


Starting epoch 4 of 10
Test Loss: 1.5733456196812485
Test Accuracy: 18.776077885952713 %
Train Loss: 1.5739595744947206
Train Accuracy: 18.738404452690165 %


Starting epoch 5 of 10
Test Loss: 1.5725049518089842
Test Accuracy: 18.636995827538247 %
Train Loss: 1.573019074588614
Train 

In [9]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 20.58414464534075%


## Part A ( ii ) Continued: A Better NumPy CNN Using Leaky ReLu Instead of Sigmoid

In [10]:
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()
import pickle

class CNN:

    def __init__(self, num_kernels, pool_size, out_nodes, seed):
        self.kernels = self.Kernels(num_kernels)  # 8x8x1  to 6x6x10
        # self.pool = self.Pool(pool_size)  # 6x6x10 to 3x3x10
        self.relu = self.ReLu(6 * 6 * num_kernels, out_nodes)  # 6x6x10 to 10
        np.random.seed(seed)

    class Kernels:
        # The set of kernels and kernel functions

        def __init__(self, num_kernels):
            self.num_kernels = num_kernels
            self.size_kernel = 3
            self.kernels = np.random.randn(num_kernels, 3, 3) / (self.size_kernel ** 2)

        def forward(self, x):
            self.cur_x = x

            height, width = x.shape
            result = np.zeros((height - 2, width - 2, self.num_kernels))

            for i in range(height - 2):
                for j in range(width - 2):
                    block = x[i:(i + 3), j:(j + 3)]
                    result[i, j] = np.sum(block * self.kernels, axis=(1, 2))
            return result

        def back(self, delta_cnn, alpha):
            height, width = self.cur_x.shape
            dk = np.zeros(self.kernels.shape)
            delta_cnn = delta_cnn.reshape(height - 2, width - 2, self.num_kernels)
            for i in range(height - 2):
                for j in range(width - 2):
                    block_x = self.cur_x[i:(i + 3), j:(j + 3)]
                    # block_delta = delta_cnn[i:(i+3), j:(j+3)]
                    for k in range(self.num_kernels):
                        dk[k] += delta_cnn[i, j, k] * block_x

            self.kernels += -alpha * dk

    class Pool:
        # use max pooling with size 2x2
        def __init__(self, pool_size):
            self.dim = pool_size  # size of the pool

        def forward(self, x):
            self.cur_input = x
            height, width, num_k = x.shape
            result = np.zeros((height // 2, width // 2, num_k))
            for i in range(height // 2):
                for j in range(width // 2):
                    block = x[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                    result[i, j] = np.amax(block, axis=(0, 1))
            return result

        def back(self, delta_pool):

            height, width, num_k = self.cur_input.shape
            delta_pool = delta_pool.reshape((height // 2, width // 2, num_k))
            delta_cnn = np.zeros((height, width, num_k))

            for i in range(height // 2):
                for j in range(width // 2):
                    block = self.cur_input[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                    b_h, b_w, b_k = block.shape
                    maximum = np.amax(block, axis=(0, 1))
                    for s in range(b_h):
                        for t in range(b_w):
                            for u in range(b_k):
                                if block[s, t, u] == maximum[u]:
                                    delta_cnn[i * 2 + s, j * 2 + t, u] = delta_pool[i, j, u]
            return delta_cnn

    class ReLu:
        # Use a fully connected layer using sigmoid activation

        def __init__(self, in_nodes, out_nodes):
            self.w = np.random.randn(out_nodes, in_nodes) / in_nodes
            self.b = np.zeros((out_nodes, 1))
            self.y_vector = np.zeros((1))
            self.z_cnn = np.zeros((1))
            self.a_cnn = np.zeros((1))
            self.z_out = np.zeros((1))
            self.a_out = np.zeros((1))

        def f(self, z):
            return np.where(z > 0, z, z * 0.2)

        def d_f(self, z):
            test = np.where(z>0, 1.0, 0.2)
            return np.where(z>0, 1.0, 0.2)

        def sig(self, z):
            return 1 / (1 + np.exp(-z))

        def d_sig(self, z):
            return self.sig(z) * (1 - self.sig(z))

        def loss(self, y, yhat):
            self.y_vector = np.zeros((len(yhat), 1))
            self.y_vector[y, 0] = 1.0
            loss = np.linalg.norm(self.y_vector - yhat)
            accuracy = 1 if np.argmax(yhat[:, 0]) == y else 0
            return loss, accuracy

        def forward(self, x):
            # In Shape: (16, 16, 18)
            self.z_cnn = x.reshape(-1, 1)
            # Reshape: (4608, 1)
            self.a_cnn = self.f(self.z_cnn)

            self.z_out = np.matmul(self.w, self.a_cnn) + self.b
            # Product Shape: (10,1)
            self.a_out = self.f(self.z_out)

            # Out Shape: (10,1)
            return self.a_out

        def back(self, alpha):

            w_grad = np.zeros(self.w.shape)
            b_grad = np.zeros(self.b.shape)

            # get delta for out layer
            delta_out = -(self.y_vector - self.a_out) * self.d_f(self.z_out)

            # get delta for hidden layer
            delta_pool = np.dot(self.w.T, delta_out) * self.d_f(self.z_cnn)

            w_grad += -alpha * np.matmul(delta_out, self.a_cnn.T)
            b_grad += -alpha * delta_out

            return delta_pool


def format_data(in_data):
    X = in_data['data']
    y = in_data['target']

    class Scaler():
        def __init__(self, training_data):
            self.u = np.mean(training_data)
            self.s = np.std(training_data)

        def scale_data(self, in_data):
            return (in_data - self.u) / self.s

    X_scale = Scaler(X)
    X = X_scale.scale_data(X)

    def split(in_X, in_y, percentage_test):
        data_size = len(in_y)
        order = np.arange(0, data_size, dtype=int)
        np.random.shuffle(order)
        out_X = in_X[order]
        out_y = np.array(in_y)[order].tolist()
        split = int(np.rint(data_size * percentage_test))
        return out_X[split:], out_X[0:split], out_y[split:], out_y[0:split]

    # Split the data into training and test set.  60% training and %40 test
    X_train, X_test, y_train, y_test = split(X, y, 0.4)

    # Reshape X_train and X_test to be (-1, 8, 8)
    X_train = X_train.reshape((-1, 8, 8))
    X_test = X_test.reshape((-1, 8, 8))
    return (X_train, X_test, y_train, y_test)


def shuffle(X, y):
    data_size = len(y)
    order = np.arange(0, data_size, dtype=int)
    np.random.shuffle(order)
    new_order = np.random.permutation(len(y))
    X = X[new_order]
    y = np.array(y)[new_order].tolist()
    return X, y


def forward(x, y, model):
    result = model.kernels.forward(x)
    # result = model.pool.forward(result)
    yhat = model.relu.forward(result)
    loss, accuracy = model.relu.loss(y, yhat)
    return yhat, loss, accuracy


def train(x, y, model, alpha=0.1):
    # forward
    _, loss, accuracy = forward(x, y, model)

    # back
    delta = model.relu.back(alpha)
    # delta = model.pool.back(delta)
    model.kernels.back(delta, alpha)

    return loss, accuracy


def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch': range(1, len(loss_test_seq) + 1),
        'train_loss': loss_train_seq,
        'test_loss': loss_test_seq,
    })

    p = figure(title='MNIST Loss: With NumPy ReLu CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train",
           source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss'

    p.add_tools(HoverTool(
        tooltips=[
            ('Epochs', '@epoch{int}'),
            ('Training Loss', '@train_loss{0.000 a}'),
            ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch': range(1, len(acc_train_seq) + 1),
        'train_acc': np.array(acc_train_seq),
        'test_acc': np.array(acc_test_seq),
    })

    p = figure(title='MNIST Accuracy: With Numpy ReLu CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
            ('Epochs', '@epoch{int}'),
            ('Training Accuracy', '@{train_acc}{%0.2f}'),
            ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)


kernel_layers = 64
pool_size = 2
epochs = epochs
out_nodes = 10
alpha = 0.4
seed = 34


def train_cnn(in_data, epochs, alpha, kernel_layers, pool_size, out_nodes, seed):
    X_train, X_test, y_train, y_test = format_data(in_data)

    model = CNN(kernel_layers, pool_size, out_nodes, seed)

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []
    train_examples = len(y_train)
    test_examples = len(y_test)

    print('=' * 30)
    print('==== Hyperparameters ====')
    print('=' * 30)
    print('Learning Rate: {}'.format(alpha))
    print('Kernel Layers: {}'.format(kernel_layers))
    # print('Pool Size: {}'.format(pool_size))
    print('Epochs: {}'.format(epochs))
    print('Seed: {}'.format(seed))
    print('=' * 30 + '\n')
    for epoch in range(epochs):

        if epoch != 0:
            print('Test Loss: {}\nTest Accuracy: {} %'.format(loss_test_seq[-1], acc_test_seq[-1] * 100))
            print('Train Loss: {}\nTrain Accuracy: {} %\n\n'.format(loss_train_seq[-1], acc_train_seq[-1] * 100))
        print('=' * 30)
        print('Starting epoch {} of {}'.format(epoch, epochs))
        X_train, y_train = shuffle(X_train, y_train)

        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0

        for i, (image, label) in enumerate(zip(X_train, y_train)):
            loss, acc = train(image, label, model, alpha)
            loss_train += loss
            acc_train += acc

        for i, (image, label) in enumerate(zip(X_test, y_test)):
            _, loss, acc = forward(image, label, model)
            loss_test += loss
            acc_test += acc

        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)

    return loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq

# % time
result_mnist_cnn_numpy_relu = train_cnn(digits, epochs, alpha, kernel_layers, pool_size, out_nodes, seed)
loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_mnist_cnn_numpy_relu



==== Hyperparameters ====
Learning Rate: 0.4
Kernel Layers: 64
Epochs: 10
Seed: 34

Starting epoch 0 of 10
Test Loss: 0.960866879506136
Test Accuracy: 44.92350486787204 %
Train Loss: 0.980784044424626
Train Accuracy: 30.519480519480517 %


Starting epoch 1 of 10
Test Loss: 0.9304082134740115
Test Accuracy: 51.599443671766345 %
Train Loss: 0.9457434569682865
Train Accuracy: 44.063079777365495 %


Starting epoch 2 of 10
Test Loss: 0.9080653396800977
Test Accuracy: 62.7260083449235 %
Train Loss: 0.9185692653755274
Train Accuracy: 58.16326530612245 %


Starting epoch 3 of 10
Test Loss: 0.8905036001193564
Test Accuracy: 66.75938803894297 %
Train Loss: 0.8979475147060755
Train Accuracy: 67.99628942486086 %


Starting epoch 4 of 10
Test Loss: 0.8757161162375555
Test Accuracy: 71.07093184979138 %
Train Loss: 0.8810611519733486
Train Accuracy: 73.46938775510205 %


Starting epoch 5 of 10
Test Loss: 0.8627323353766468
Test Accuracy: 74.54798331015299 %
Train Loss: 0.8665579867651354
Train Accura

In [11]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 80.11126564673157%


## Part B ( i ) Baseline PyTorch Neural Network Without Convolution on CIFAR-10 Dataset II

In [12]:
# Import Libraries
from sklearn.metrics import accuracy_score #
import numpy as np
import numpy.random as r # We will randomly initialize our weights
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()
import pickle

if __name__ == '__main__':



    # Import modules
    import torch
    import pickle
    import numpy as np
    import torch.nn.functional as F
    from torch import nn
    from torch import optim
    from torch.utils.data import Dataset, DataLoader
    from bokeh.plotting import figure, output_file, show
    from bokeh.models import ColumnDataSource, HoverTool
    from bokeh.models.formatters import NumeralTickFormatter
    from bokeh.io import output_notebook
    output_notebook()

    # TODO Load Dataset
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    class MNISTDataset(Dataset):
        def __init__(self, data, label):
            self.data = data.reshape((-1, 32, 32, 3))
            self.label = label

        def __len__(self):
            return len(self.label)

        def __getitem__(self, item):
            # swap color axis because
            # numpy image: H x W x C
            # torch image: C X H X W
            image = self.data[item].transpose((2, 0, 1))
            image = torch.from_numpy(image)
            target = self.label[item]
            target = torch.from_numpy(target)
            return (image, target)


    class MNIST(nn.Module):

        # Our batch shape for input x is (1, 8, 8)

        def __init__(self):
            super(MNIST, self).__init__()

            # Input channels = 1, output channels = 18
            # self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
            # self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

            # self.fc1 = nn.Linear(18 * 8 * 8, 64)
            self.fc1 = nn.Linear(3072, 30)
            self.fc2 = nn.Linear(30, 10)

        def forward(self, x):
            # Activation of the first convolution
            # Size changes from (1, 8, 8) to (18, 8, 8)
            # x = F.relu(self.conv1(x))

            # x = self.pool(x)

            # Reshape data to input to the input layer of the neural net
            # Size changes from (18, 16, 16) to (1, 1152)
            x = x.view(-1, 32 * 32 * 3)

            # Computes the activation of the first fully connected layer
            # Size changes from (1, 1152) to (1, 64)
            x = torch.sigmoid(self.fc1(x))
            x = torch.sigmoid(self.fc2(x))

            # x = self.fc2(x)
            # x = F.log_softmax(x, dim=1)

            return x


    def scorecard(output, labels):
        score = 0
        score = np.where(np.argmax(output, axis=1) == np.argmax(labels, axis=1), 1, 0)
        # for i in range(len(output)):
        #     if np.argmax(output[i,:], axis=1) == np.argmax(labels[i,:], axis=1):
        #         score += 1
        score = np.sum(score)
        return score

    def trainModel(model, batch_size, num_epochs, learning_rate):
        print("===== HYPERPARAMETERS =====")
        print("batch_size=", batch_size)
        print("epochs=", num_epochs)
        print("learning_rate=", learning_rate)
        print("=" * 30)

        X = cifar[b'data']
        y = cifar[b'labels']

        class Scaler():
          def __init__(self, training_data):
            self.u = np.mean(training_data)
            self.s = np.std(training_data)

          def scale_data(self, in_data):
            return (in_data - self.u) / self.s

        X_scale = Scaler(X)
        X = X_scale.scale_data(X)

        def split(in_X, in_y, percentage_test):
            data_size = len(in_y)
            order = np.arange(0, data_size, dtype=int)
            np.random.shuffle(order)
            out_X = in_X[order]
            out_y = np.array(in_y)[order].tolist()
            split = int(np.rint(data_size * percentage_test))
            return out_X[split:], out_X[0:split], out_y[split:], out_y[0:split]

        # Split the data into training and test set.  60% training and %40 test
        X_train, X_test, y_train, y_test = split(X, y, 0.4)

        def convert_y_to_vect(y):
            y_vect = np.zeros((len(y), 10))
            for i in range(len(y)):
                y_vect[i, y[i]] = 1
            return y_vect
        
        # convert digits to vectors
        y_v_train = convert_y_to_vect(y_train)
        y_v_test = convert_y_to_vect(y_test)

        train_dataset = MNISTDataset(X_train, y_v_train)
        test_dataset = MNISTDataset(X_test, y_v_test)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        torch.manual_seed(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(0)
        mean, std = (0.5,), (0.5,)

        criterion = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

        model = model.double()

        loss_train_seq = []
        loss_test_seq = []
        acc_train_seq = []
        acc_test_seq = []

        for i in range(num_epochs):
            loss_train = 0.0
            loss_test = 0.0
            acc_train = 0.0
            acc_test = 0.0
            train_examples = 0
            test_examples = 0

            for j, (images, labels) in enumerate(train_loader, 0):
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels)
                acc_train += scorecard(output.detach().numpy(), labels.detach().numpy())
                train_examples += len(labels)
                loss_train += loss.detach().numpy()
                loss.backward()
                optimizer.step()

            for images, labels in test_loader:
                with torch.no_grad():
                    output = model(images)

                    loss = criterion(output, labels)
                    loss_test += loss.detach().numpy()
                    acc_test += scorecard(output.detach().numpy(), labels.detach().numpy())
                    test_examples += len(labels)
            loss_train_seq.append(loss_train / train_examples)
            loss_test_seq.append(loss_test / test_examples)
            acc_train_seq.append(acc_train / train_examples)
            acc_test_seq.append(acc_test / test_examples)
            print('Epoch {} with test loss {} and test accuracy {}\n\n'.format(i, (loss_test / test_examples),
                                                                               (acc_test / test_examples)))

        return (loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)


    def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
        print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
        source = ColumnDataSource(data={
            'epoch': range(1, len(loss_test_seq) + 1),
            'train_loss': loss_train_seq,
            'test_loss': loss_test_seq,
        })

        p = figure(title='CIFAR Loss: Baseline No CNN', plot_width=400, plot_height=400)

        p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train",
               source=source)
        p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test",
               source=source)
        p.legend.location = "top_right"
        p.xaxis.axis_label = 'Epochs'
        p.yaxis.axis_label = 'Loss'

        p.add_tools(HoverTool(
            tooltips=[
                ('Epochs', '@epoch{int}'),
                ('Training Loss', '@train_loss{0.000 a}'),
                ('Test Loss', '@test_loss{0.000 a}'),
            ],

            mode='mouse'
        ))

        show(p)

        source = ColumnDataSource(data={
            'epoch': range(1, len(acc_train_seq) + 1),
            'train_acc': np.array(acc_train_seq),
            'test_acc': np.array(acc_test_seq),
        })

        p = figure(title='CIFAR Accuracy: Baseline No CNN', plot_width=400, plot_height=400)

        p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train",
               source=source)
        p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test",
               source=source)
        p.legend.location = "bottom_right"
        p.xaxis.axis_label = 'Epochs'
        p.yaxis.axis_label = 'Accuracy'
        p.yaxis.formatter = NumeralTickFormatter(format='0 %')

        p.add_tools(HoverTool(
            tooltips=[
                ('Epochs', '@epoch{int}'),
                ('Training Accuracy', '@{train_acc}{%0.2f}'),
                ('Test Accuracy', '@{test_acc}{%0.2f}'),
            ],

            mode='mouse'
        ))

        show(p)


    batch_size = 1
    learning_rate = 0.01
    num_epochs = epochs

    model = MNIST()

    % time result_baseline_pytorch_cifar = trainModel(model, batch_size, num_epochs, learning_rate)
    loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_baseline_pytorch_cifar





===== HYPERPARAMETERS =====
batch_size= 1
epochs= 10
learning_rate= 0.01
Epoch 0 with test loss 0.13411513558932125 and test accuracy 0.1825


Epoch 1 with test loss 0.10683383299924283 and test accuracy 0.2275


Epoch 2 with test loss 0.09804264539014121 and test accuracy 0.255


Epoch 3 with test loss 0.09406389268382308 and test accuracy 0.275


Epoch 4 with test loss 0.0918686678808481 and test accuracy 0.275


Epoch 5 with test loss 0.09049231031483755 and test accuracy 0.2875


Epoch 6 with test loss 0.08954662704477714 and test accuracy 0.285


Epoch 7 with test loss 0.08885028867194153 and test accuracy 0.29


Epoch 8 with test loss 0.08830935536316323 and test accuracy 0.2825


Epoch 9 with test loss 0.08787112842353101 and test accuracy 0.2825


CPU times: user 22.7 s, sys: 1.01 s, total: 23.7 s
Wall time: 8.32 s


In [13]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 28.249999999999996%


## Part B ( ii ) Extended Algorithm using CNN with Sigmoid on Dataset II

In [14]:
import numpy as np
import pickle
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook
output_notebook()

# def unpickle(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict

# cifar = unpickle('/home/justin/Documents/00_NYU/02_Classes/03_2020_Spring/ML/ML-Project/NYU-ML-Project/data/cifar-10.p')
# print('cifar data shape {}'.format(cifar[b'data']))

class CNN:

    def __init__(self, num_kernels, out_nodes, seed):
        self.kernels = self.Kernels(num_kernels)  # 8x8x1  to 6x6x10
        # self.pool = self.Pool(pool_size)  # 6x6x10 to 3x3x10
        self.sigmoid = self.Sigmoid(30 * 30 * num_kernels, out_nodes)  # 6x6x10 to 10
        np.random.seed(seed)

    class Kernels:
        # The set of kernels and kernel functions

        def __init__(self, num_kernels):
            self.num_kernels = num_kernels
            self.size_kernel = 3
            self.kernels = np.random.randn(num_kernels, 3, 3) / (self.size_kernel ** 2)
            self.k_per_c = 0
            self.in_channels = 0
            self.cur_x = np.zeros((1))

        def forward(self, x):
            self.cur_x = x

            height, width, self.in_channels = x.shape
            self.k_per_c = self.num_kernels // self.in_channels
            result = np.zeros((30,30,self.num_kernels))  # assuming valid. start with bias values

            for c in range(0, self.in_channels):
                for i in range(height-2):
                    for j in range(width-2):
                        block = np.array([x[i:(i+3), j:(j+3), c]])
                        result[i,j,c*self.k_per_c: (c+1)*self.k_per_c] = np.sum(block * self.kernels[c*self.k_per_c: (c+1)*self.k_per_c], axis=(1,2))
            return result

        def back(self, delta_cnn, alpha):
            height, width, channels = self.cur_x.shape
            dk = np.zeros(self.kernels.shape)
            delta_cnn = delta_cnn.reshape(height-2, width-2,self.num_kernels)
            for c in range(channels):
                for i in range(height - 2):
                    for j in range(width - 2):
                        block_x = self.cur_x[i:(i + self.size_kernel), j:(j + self.size_kernel), c]
                        block_x = np.ones((self.k_per_c, self.size_kernel, self.size_kernel)) * block_x
                        block_delta = delta_cnn[i, j, (c*self.k_per_c):(c*self.k_per_c+self.k_per_c)].reshape((self.k_per_c,1,1))
                        dk[c] += np.sum(block_delta * block_x, axis=0)

            self.kernels += -alpha * dk

    class Sigmoid:
        # Use a fully connected layer using sigmoid activation

        def __init__(self, in_nodes, out_nodes):
            self.w = np.random.randn(out_nodes, in_nodes) / in_nodes
            self.b = np.zeros((out_nodes, 1))

        def sig(self,z):
            return 1 / (1+np.exp(-z))

        def d_sig(self,z):
            return self.sig(z) * (1-self.sig(z))

        def loss(self, y, yhat):
            self.y_vector = np.zeros((len(yhat), 1))
            self.y_vector[y, 0] = 1.0
            loss = np.linalg.norm(self.y_vector - yhat)
            accuracy = 1 if np.argmax(yhat[:, 0]) == y else 0
            return loss, accuracy

        def forward(self, x):
            # TODO forward pass
            self.z_cnn = x.reshape(-1, 1)
            self.a_cnn = self.sig(self.z_cnn)

            self.z_out = np.matmul(self.w, self.a_cnn) + self.b
            self.a_out = self.sig(self.z_out)

            return self.a_out

        def back(self, alpha):

            w_grad = np.zeros(self.w.shape)
            b_grad = np.zeros(self.b.shape)

            # get delta for out layer
            delta_out = -(self.y_vector - self.a_out) * self.d_sig(self.z_out)

            # get delta for hidden layer
            delta_pool = np.dot(self.w.T, delta_out) * self.d_sig(self.z_cnn)

            w_grad += -alpha * np.matmul(delta_out, self.a_cnn.T)
            b_grad += -alpha * delta_out
            # TODO back pass through pool and cnn layers
            return delta_pool



def format_data(in_data):
    X = in_data[b'data']
    y = in_data[b'labels']
    print('X shape {}'.format(X.shape))
    print('y length {}'.format(len(y)))

    class Scaler():
      def __init__(self, training_data):
        self.u = np.mean(training_data)
        self.s = np.std(training_data)

      def scale_data(self, in_data):
        return (in_data - self.u) / self.s

    X_scale = Scaler(X)
    X = X_scale.scale_data(X)

    def split(in_X, in_y, percentage_test):
        data_size = len(in_y)
        order = np.arange(0, data_size, dtype=int)
        np.random.shuffle(order)
        out_X = in_X[order]
        out_y = np.array(in_y)[order].tolist()
        split = int(np.rint(data_size * percentage_test))
        return in_X[split:], in_X[0:split], in_y[split:], in_y[0:split]

    # Split the data into training and test set.  60% training and %40 test
    X_train, X_test, y_train, y_test = split(X, y, 0.4)

    # Reshape X_train and X_test to be (-1, 32, 32, 3)
    X_train = X_train.reshape((-1, 32, 32, 3))
    X_test = X_test.reshape((-1, 32, 32, 3))
    return (X_train, X_test, y_train, y_test)

def shuffle(X, y):
    data_size = len(y)
    order = np.arange(0, data_size, dtype=int)
    np.random.shuffle(order)
    new_order = np.random.permutation(len(y))
    X = X[new_order]
    y = np.array(y)[new_order].tolist()
    return X, y

def forward(x, y, model):
    result = model.kernels.forward(x)
    # result = model.pool.forward(result)
    yhat = model.sigmoid.forward(result)
    loss, accuracy = model.sigmoid.loss(y, yhat)
    return yhat, loss, accuracy


def train(x, y, model, alpha=0.1):
    # forward
    _, loss, accuracy = forward(x, y, model)

    # back
    delta = model.sigmoid.back(alpha)
    # delta = model.pool.back(delta)
    model.kernels.back(delta, alpha)

    return loss, accuracy


def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch'            : range(1, len(loss_test_seq) + 1),
        'train_loss'    : loss_train_seq,
        'test_loss'        :  loss_test_seq,
    })

    p = figure(title='CIFAR Loss: With NumPy Sigmoid CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss'

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Loss', '@train_loss{0.000 a}'),
                  ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_train_seq) + 1),
        'train_acc'    : np.array(acc_train_seq),
        'test_acc'        :  np.array(acc_test_seq),
    })

    p = figure(title='CIFAR Accuracy: With Numpy Sigmoid CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8 , line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('Training Accuracy', '@{train_acc}{%0.2f}'),
                  ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)

# make this a number divisible by # of channels (3 in this case)
kernel_layers = 9
pool_size = 2
epochs = epochs
out_nodes = 10
alpha = 0.3
seed = 0


def train_cnn(in_data, epochs, alpha, kernel_layers, out_nodes, seed):
    X_train, X_test, y_train, y_test = format_data(in_data)

    model = CNN(kernel_layers, out_nodes, seed)

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []
    train_examples = len(y_train)
    test_examples = len(y_test)

    print('=' * 30)
    print('==== Hyperparameters ====')
    print('=' * 30)
    print('Learning Rate: {}'.format(alpha))
    print('Kernel Layers: {}'.format(kernel_layers))
    print('Epochs: {}'.format(epochs))
    print('Seed: {}'.format(seed))
    print('=' * 30 + '\n')
    for epoch in range(epochs):
        print('=' * 30)
        print('Starting epoch {} of {}'.format(epoch, epochs))
        X_train, y_train = shuffle(X_train, y_train)

        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0

        for i, (image, label) in enumerate(zip(X_train, y_train)):
            loss, acc = train(image, label, model, alpha)
            loss_train += loss
            acc_train += acc
            # print('Image {} of {}'.format(i+1, len(y_train)))
            # print('Train Loss: {}\nTrain Accuracy: {} %\n\n'.format(loss, acc * 100))

        for i, (image, label) in enumerate(zip(X_test, y_test)):
            _, loss, acc = forward(image, label, model)
            loss_test += loss
            acc_test += acc

        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)
        print('Test Loss: {}\nTest Accuracy: {} %'.format(loss_test_seq[-1], acc_test_seq[-1] * 100))
        print('Train Loss: {}\nTrain Accuracy: {} %\n\n'.format(loss_train_seq[-1], acc_train_seq[-1] * 100))

    return loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq

# % time
% time result_cifar_cnn_numpy = train_cnn(cifar, epochs, alpha, kernel_layers, out_nodes, seed)
loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_cifar_cnn_numpy



X shape (1000, 3072)
y length 1000
==== Hyperparameters ====
Learning Rate: 0.3
Kernel Layers: 9
Epochs: 10
Seed: 0

Starting epoch 0 of 10
Test Loss: 1.581666096462263
Test Accuracy: 8.0 %
Train Loss: 1.5816199067428032
Train Accuracy: 11.0 %


Starting epoch 1 of 10
Test Loss: 1.5816628200734468
Test Accuracy: 8.0 %
Train Loss: 1.5816168563347772
Train Accuracy: 11.0 %


Starting epoch 2 of 10
Test Loss: 1.5816595473690485
Test Accuracy: 7.5 %
Train Loss: 1.5816118847847858
Train Accuracy: 10.666666666666668 %


Starting epoch 3 of 10
Test Loss: 1.5816563928838914
Test Accuracy: 7.249999999999999 %
Train Loss: 1.5816099750802286
Train Accuracy: 10.333333333333334 %


Starting epoch 4 of 10
Test Loss: 1.5816531512085887
Test Accuracy: 7.000000000000001 %
Train Loss: 1.5816057920608109
Train Accuracy: 10.166666666666666 %


Starting epoch 5 of 10
Test Loss: 1.581649842367293
Test Accuracy: 7.000000000000001 %
Train Loss: 1.5816019444440321
Train Accuracy: 10.0 %


Starting epoch 6 of 1

In [15]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 5.25%


## Part B ( ii ) Continued a Better Extended Algorithm using CNN with Leaky ReLu on Dataset II


In [16]:
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook

output_notebook()
import pickle


# def unpickle(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict

# cifar = unpickle('/home/justin/Documents/00_NYU/02_Classes/03_2020_Spring/ML/ML-Project/NYU-ML-Project/data/cifar-10.p')
# print('cifar data shape {}'.format(cifar[b'data']))

# epochs = 10

class CNN:

    def __init__(self, num_kernels, pool_size, out_nodes, seed):
        self.kernels = self.Kernels(num_kernels)  # 8x8x1  to 6x6x10
        # self.pool = self.Pool(pool_size)  # 6x6x10 to 3x3x10
        self.relu = self.ReLu(30 * 30 * num_kernels, out_nodes)  # 6x6x10 to 10
        np.random.seed(seed)

    class Kernels:
        # The set of kernels and kernel functions

        def __init__(self, num_kernels):
            self.num_kernels = num_kernels
            self.size_kernel = 3
            self.kernels = np.random.randn(num_kernels, 3, 3) / (self.size_kernel ** 2)

        def forward(self, x):
            self.cur_x = x

            height, width = x.shape
            result = np.zeros((height - 2, width - 2, self.num_kernels))

            for i in range(height - 2):
                for j in range(width - 2):
                    block = x[i:(i + 3), j:(j + 3)]
                    result[i, j] = np.sum(block * self.kernels, axis=(1, 2))
            return result

        def back(self, delta_cnn, alpha):
            height, width = self.cur_x.shape
            dk = np.zeros(self.kernels.shape)
            delta_cnn = delta_cnn.reshape(height - 2, width - 2, self.num_kernels)
            for i in range(height - 2):
                for j in range(width - 2):
                    block_x = self.cur_x[i:(i + 3), j:(j + 3)]
                    # block_delta = delta_cnn[i:(i+3), j:(j+3)]
                    for k in range(self.num_kernels):
                        dk[k] += delta_cnn[i, j, k] * block_x

            self.kernels += -alpha * dk

    class Pool:
        # use max pooling with size 2x2
        def __init__(self, pool_size):
            self.dim = pool_size  # size of the pool

        def forward(self, x):
            self.cur_input = x
            height, width, num_k = x.shape
            result = np.zeros((height // 2, width // 2, num_k))
            for i in range(height // 2):
                for j in range(width // 2):
                    block = x[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                    result[i, j] = np.amax(block, axis=(0, 1))
            return result

        def back(self, delta_pool):

            height, width, num_k = self.cur_input.shape
            delta_pool = delta_pool.reshape((height // 2, width // 2, num_k))
            delta_cnn = np.zeros((height, width, num_k))

            for i in range(height // 2):
                for j in range(width // 2):
                    block = self.cur_input[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                    b_h, b_w, b_k = block.shape
                    maximum = np.amax(block, axis=(0, 1))
                    for s in range(b_h):
                        for t in range(b_w):
                            for u in range(b_k):
                                if block[s, t, u] == maximum[u]:
                                    delta_cnn[i * 2 + s, j * 2 + t, u] = delta_pool[i, j, u]
            return delta_cnn

    class ReLu:
        # Use a fully connected layer using sigmoid activation

        def __init__(self, in_nodes, out_nodes):
            self.w = np.random.randn(out_nodes, in_nodes) / in_nodes
            self.b = np.zeros((out_nodes, 1))
            self.y_vector = np.zeros((1))
            self.z_cnn = np.zeros((1))
            self.a_cnn = np.zeros((1))
            self.z_out = np.zeros((1))
            self.a_out = np.zeros((1))

        def f(self, z):
            return np.where(z > 0, z, z * 0.2)

        def d_f(self, z):
            test = np.where(z>0, 1.0, 0.2)
            return np.where(z>0, 1.0, 0.2)

        def sig(self, z):
            return 1 / (1 + np.exp(-z))

        def d_sig(self, z):
            return self.sig(z) * (1 - self.sig(z))

        def loss(self, y, yhat):
            self.y_vector = np.zeros((len(yhat), 1))
            self.y_vector[y, 0] = 1.0
            loss = np.linalg.norm(self.y_vector - yhat)
            accuracy = 1 if np.argmax(yhat[:, 0]) == y else 0
            return loss, accuracy

        def forward(self, x):
            # In Shape: (16, 16, 18)
            self.z_cnn = x.reshape(-1, 1)
            # Reshape: (4608, 1)
            self.a_cnn = self.f(self.z_cnn)

            self.z_out = np.matmul(self.w, self.a_cnn) + self.b
            # Product Shape: (10,1)
            self.a_out = self.f(self.z_out)

            # Out Shape: (10,1)
            return self.a_out

        def back(self, alpha):

            w_grad = np.zeros(self.w.shape)
            b_grad = np.zeros(self.b.shape)

            # get delta for out layer
            delta_out = -(self.y_vector - self.a_out) * self.d_f(self.z_out)

            # get delta for hidden layer
            delta_pool = np.dot(self.w.T, delta_out) * self.d_f(self.z_cnn)

            w_grad += -alpha * np.matmul(delta_out, self.a_cnn.T)
            b_grad += -alpha * delta_out

            return delta_pool


def format_data(in_data):
    X = in_data[b'data']
    y = in_data[b'labels']

    X = np.sum(X.reshape((-1, 32, 32, 3)), axis=3)
    X = X.reshape(-1, 32*32)

    class Scaler():
        def __init__(self, training_data):
            self.u = np.mean(training_data)
            self.s = np.std(training_data)

        def scale_data(self, in_data):
            return (in_data - self.u) / self.s

    X_scale = Scaler(X)
    X = X_scale.scale_data(X)

    def split(in_X, in_y, percentage_test):
        data_size = len(in_y)
        order = np.arange(0, data_size, dtype=int)
        np.random.shuffle(order)
        out_X = in_X[order]
        out_y = np.array(in_y)[order].tolist()
        split = int(np.rint(data_size * percentage_test))
        return out_X[split:], out_X[0:split], out_y[split:], out_y[0:split]

    # Split the data into training and test set.  60% training and %40 test
    X_train, X_test, y_train, y_test = split(X, y, 0.4)

    # Reshape X_train and X_test to be (-1, 8, 8)
    X_train = X_train.reshape((-1, 32, 32))
    X_test = X_test.reshape((-1, 32, 32))
    return (X_train, X_test, y_train, y_test)


def shuffle(X, y):
    data_size = len(y)
    order = np.arange(0, data_size, dtype=int)
    np.random.shuffle(order)
    new_order = np.random.permutation(len(y))
    X = X[new_order]
    y = np.array(y)[new_order].tolist()
    return X, y


def forward(x, y, model):
    result = model.kernels.forward(x)
    # result = model.pool.forward(result)
    yhat = model.relu.forward(result)
    loss, accuracy = model.relu.loss(y, yhat)
    return yhat, loss, accuracy


def train(x, y, model, alpha=0.1):
    # forward
    _, loss, accuracy = forward(x, y, model)

    # back
    delta = model.relu.back(alpha)
    # delta = model.pool.back(delta)
    model.kernels.back(delta, alpha)

    return loss, accuracy


def plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq):
    print('The test set prediction accuracy is {}%'.format(acc_test_seq[-1] * 100))
    source = ColumnDataSource(data={
        'epoch': range(1, len(loss_test_seq) + 1),
        'train_loss': loss_train_seq,
        'test_loss': loss_test_seq,
    })

    p = figure(title='MNIST Loss With NumPy ReLu CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_loss', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train",
           source=source)
    p.line(x='epoch', y='test_loss', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "top_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Loss'

    p.add_tools(HoverTool(
        tooltips=[
            ('Epochs', '@epoch{int}'),
            ('Training Loss', '@train_loss{0.000 a}'),
            ('Test Loss', '@test_loss{0.000 a}'),
        ],

        mode='mouse'
    ))

    show(p)

    source = ColumnDataSource(data={
        'epoch': range(1, len(acc_train_seq) + 1),
        'train_acc': np.array(acc_train_seq),
        'test_acc': np.array(acc_test_seq),
    })

    p = figure(title='MNIST Accuracy With Numpy ReLu CNN', plot_width=400, plot_height=400)

    p.line(x='epoch', y='train_acc', color='#329fe3', line_alpha=0.8, line_width=2, legend_label="Train", source=source)
    p.line(x='epoch', y='test_acc', color='#e33270', line_alpha=0.8, line_width=2, legend_label="Test", source=source)
    p.legend.location = "bottom_right"
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')

    p.add_tools(HoverTool(
        tooltips=[
            ('Epochs', '@epoch{int}'),
            ('Training Accuracy', '@{train_acc}{%0.2f}'),
            ('Test Accuracy', '@{test_acc}{%0.2f}'),
        ],

        mode='mouse'
    ))

    show(p)


kernel_layers = 64
pool_size = 2
epochs = epochs
out_nodes = 10
alpha = 0.4
seed = 34


def train_cnn(in_data, epochs, alpha, kernel_layers, pool_size, out_nodes, seed):
    X_train, X_test, y_train, y_test = format_data(in_data)

    model = CNN(kernel_layers, pool_size, out_nodes, seed)

    loss_train_seq = []
    loss_test_seq = []
    acc_train_seq = []
    acc_test_seq = []
    train_examples = len(y_train)
    test_examples = len(y_test)

    print('=' * 30)
    print('==== Hyperparameters ====')
    print('=' * 30)
    print('Learning Rate: {}'.format(alpha))
    print('Kernel Layers: {}'.format(kernel_layers))
    # print('Pool Size: {}'.format(pool_size))
    print('Epochs: {}'.format(epochs))
    print('Seed: {}'.format(seed))
    print('=' * 30 + '\n')
    for epoch in range(epochs):

        if epoch != 0:
            print('Test Loss: {}\nTest Accuracy: {} %'.format(loss_test_seq[-1], acc_test_seq[-1] * 100))
            print('Train Loss: {}\nTrain Accuracy: {} %\n\n'.format(loss_train_seq[-1], acc_train_seq[-1] * 100))
        print('=' * 30)
        print('Starting epoch {} of {}'.format(epoch, epochs))
        X_train, y_train = shuffle(X_train, y_train)

        loss_train = 0.0
        loss_test = 0.0
        acc_train = 0.0
        acc_test = 0.0

        for i, (image, label) in enumerate(zip(X_train, y_train)):
            loss, acc = train(image, label, model, alpha)
            loss_train += loss
            acc_train += acc

        for i, (image, label) in enumerate(zip(X_test, y_test)):
            _, loss, acc = forward(image, label, model)
            loss_test += loss
            acc_test += acc

        loss_train_seq.append(loss_train / train_examples)
        loss_test_seq.append(loss_test / test_examples)
        acc_train_seq.append(acc_train / train_examples)
        acc_test_seq.append(acc_test / test_examples)

    return loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq

# % time
%time result_cifar_cnn_numpy = train_cnn(cifar, epochs, alpha, kernel_layers, pool_size, out_nodes, seed)
loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq = result_cifar_cnn_numpy



==== Hyperparameters ====
Learning Rate: 0.4
Kernel Layers: 64
Epochs: 10
Seed: 34

Starting epoch 0 of 10
Test Loss: 0.9991970552558755
Test Accuracy: 14.75 %
Train Loss: 0.999357508750234
Train Accuracy: 14.166666666666666 %


Starting epoch 1 of 10
Test Loss: 0.9989116554489762
Test Accuracy: 18.0 %
Train Loss: 0.9990764915806303
Train Accuracy: 15.5 %


Starting epoch 2 of 10
Test Loss: 0.9986015451667479
Test Accuracy: 18.0 %
Train Loss: 0.9987578294344513
Train Accuracy: 17.5 %


Starting epoch 3 of 10
Test Loss: 0.9982804678864468
Test Accuracy: 18.75 %
Train Loss: 0.9984195604574811
Train Accuracy: 17.833333333333336 %


Starting epoch 4 of 10
Test Loss: 0.9979514020143471
Test Accuracy: 19.25 %
Train Loss: 0.9980725982740304
Train Accuracy: 19.166666666666668 %


Starting epoch 5 of 10
Test Loss: 0.9976148112677707
Test Accuracy: 20.25 %
Train Loss: 0.9977185402987123
Train Accuracy: 20.166666666666664 %


Starting epoch 6 of 10
Test Loss: 0.9972725416658299
Test Accuracy: 19.

In [17]:
plot_results(loss_train_seq, loss_test_seq, acc_train_seq, acc_test_seq)

The test set prediction accuracy is 19.0%


## Synopsis: Visualizations

In [0]:
def plot_results_compare_set_one(baseline, new_benchmark):
    
    loss_train_base, loss_test_base, acc_train_base, acc_test_base = baseline
    loss_train_new, loss_test_new, acc_train_new, acc_test_new = new_benchmark

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_test_base) + 1),
        'test_acc_base'        :  np.array(acc_test_base),
        'test_acc_new'        :  np.array(acc_test_new),
    })

    p = figure(title='MNIST Accuracy: With and Without CNN', plot_width=600, plot_height=400)

    l_van = p.line(x='epoch', y='test_acc_base', color='#329fe3', line_alpha=0.8 , line_width=2, source=source)
    l_torch = p.line(x='epoch', y='test_acc_new', color='#e33270', line_alpha=0.8, line_width=2, source=source)
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')
    

    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('No CNN', '@{test_acc_base}{%0.2f}'),
                  ('NumPy CNN', '@{test_acc_new}{%0.2f}'),
        ],

        mode='mouse'
    ))

    legend = Legend(items=[('No CNN', [l_van]), ('NumPy CNN with ReLu', [l_torch]),], location=(0,250))
    p.add_layout(legend, 'left')

    show(p)




In [0]:
def plot_results_compare_set_two(baseline, new_benchmark):
    
    loss_train_base, loss_test_base, acc_train_base, acc_test_base = baseline
    loss_train_new, loss_test_new, acc_train_new, acc_test_new = new_benchmark

    source = ColumnDataSource(data={
        'epoch'            : range(1, len(acc_test_base) + 1),
        'test_acc_base'        :  np.array(acc_test_base),
        'test_acc_new'        :  np.array(acc_test_new),
    })

    p = figure(title='CIFAR Accuracy: With and Without CNN', plot_width=600, plot_height=400)

    l_van = p.line(x='epoch', y='test_acc_base', color='#329fe3', line_alpha=0.8 , line_width=2, source=source)
    l_torch = p.line(x='epoch', y='test_acc_new', color='#e33270', line_alpha=0.8, line_width=2, source=source)
    p.xaxis.axis_label = 'Epochs'
    p.yaxis.axis_label = 'Accuracy'
    p.yaxis.formatter = NumeralTickFormatter(format='0 %')
    
    p.add_tools(HoverTool(
        tooltips=[
                  ('Epochs', '@epoch{int}'),
                  ('No CNN', '@{test_acc_base}{%0.2f}'),
                  ('NumPy CNN', '@{test_acc_new}{%0.2f}'),
        ],

        mode='mouse'
    ))

    legend = Legend(items=[('No CNN', [l_van]), ('NumPy CNN with ReLu', [l_torch]),], location=(0,250))
    p.add_layout(legend, 'left')

    show(p)

## Synopsis: Compare Results on MNIST Dataset I

In [20]:
plot_results_compare_set_one(result_baseline_pytorch_mnist, result_mnist_cnn_numpy_relu)

## Synopsis: Compare Results on CIFAR-10 Dataset II

In [21]:
plot_results_compare_set_two(result_baseline_pytorch_cifar, result_cifar_cnn_numpy)